In [1]:
pwd

'C:\\Users\\Asus'

In [2]:
!pip install selenium
import selenium
from selenium import webdriver
# This is the path I use
DRIVER_PATH = '.../Desktop/Scraping/chromedriver'
# Put the path for your ChromeDriver here
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)


C:\Users\Asus\AppData\Local\Temp/ipykernel_9820/952011895.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)


In [3]:
wd.get('https://google.com')

In [21]:
from selenium.webdriver.common.by import By
search_box = wd.find_element(By.CSS_SELECTOR, 'input.gLFyf')
search_box.send_keys('Ronaldo')

MaxRetryError: HTTPConnectionPool(host='localhost', port=52351): Max retries exceeded with url: /session/c174818ca3724394acef0b5a5da42b5a/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002211C088820>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [9]:
wd.quit()

In [10]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_element(By.CSS_SELECTOR,"img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_element(By.CSS_SELECTOR,'img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element(By.CSS_SELECTOR,".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [11]:
pip install Pillow

In [12]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [13]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [17]:
import os
import time

In [18]:
search_term = 'Lionel Messi'
search_and_download(search_term=search_term,driver_path=DRIVER_PATH)

C:\Users\Asus\AppData\Local\Temp/ipykernel_9820/4098395253.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:


TypeError: object of type 'WebElement' has no len()